# Augmentation notebook

In [ ]:
import sys
sys.path.insert(1,"..")
from datasets.augmentation.randaugment import RandAugment
import matplotlib.pyplot as plt
import random
from PIL import Image
import pandas as pd
from skimage import img_as_ubyte
import os
from torchvision import transforms
import numpy as np

## 1 - Control flags and  image paths

In [ ]:
img_path="/home/gabrielemeoni/project/end2end/END2END/MSMatch/DATA/warmup_events_dataset/event/Etna_03_G1_(0,_768,_256,_1024)_A21.00.pkl" #img path
augmentation_type = "strong" #Supported: strong, weak

For strong augmentations, it is possible to use a custom augmentation policy selecting only among the possible augmentations.
To this aim, you can use `strong_augmentation_wanted` list. To use all the possible augmentations, select `strong_augmentation_wanted=["all"]`; otherwise, fill the list through the wanted augmentations. Supported augmentations are:

<ul>
    <li>"AutoContrast"</li>
    <li>"Brightness"</li>
    <li>"Color"</li>
    <li>"Contrast"</li>
    <li>"Equalize"</li>
    <li>"Identity</li>
    <li>"Posterize"</li>
    <li>"Rotate"</li>
    <li>"Sharpness"</li>
    <li>"ShearX"</li>
    <li>"ShearY"</li>
    <li>"Solarize"</li>
    <li>"TranslateX"</li>
    <li>"TranslateY"</li>
</ul>

In [ ]:
augmentations_list=["AutoContrast", "Brightness","Color","Contrast","Equalize","Identity","Posterize","Rotate","Sharpness","ShearX","ShearY","Solarize","TranslateX","TranslateY"] 

## 2- Opening and plotting the original image

In [ ]:
img = img_as_ubyte(pd.read_pickle(img_path)/4095)

In [ ]:
image_name = img_path[img_path.rfind('/')+1:img_path.rfind('.')] #finding image name
plt.title("Original image: "+ str(image_name), fontsize=14, fontweight='bold')
plt.imshow(img)

# 3. Create a function to apply augmentations

In [ ]:
def apply_augment(img, strong_augmentation_wanted):
    transform = transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.RandomHorizontalFlip(),
                transforms.RandomAffine(0,translate=(0,0.125)),
            ]
        )
    transform_to_img = transforms.ToPILImage()
    img = (transform_to_img(transform(img)))


    if augmentation_type == "strong": 
        randaugment = RandAugment(3, 5, False)
        #it might be useful to extract specific augmentation transforms from the list. 
        randaugment_dict = {"AutoContrast" : 0, "Brightness" : 1, "Color" : 2, "Contrast" : 3,"Equalize" : 4,"Identity" : 5,"Posterize" : 6,"Rotate" : 7,"Sharpness" : 8,"ShearX" : 9,"ShearY" : 10,"Solarize" : 11,"TranslateX" : 12,"TranslateY" : 13}

        if strong_augmentation_wanted[0] == "all":
            augment_list = randaugment.augment_list
        else:
            augment_list = []
            for augmentation in strong_augmentation_wanted:
                augment_list.append(randaugment.augment_list[randaugment_dict[augmentation]])

        for op, min_val, max_val in augment_list:
            val = min_val + float(max_val - min_val) * random.random()
            print(op)
            img = op(img, val)
    return img

 ## 3- Plotting and saving the augmented image

In [ ]:
plt.close()
fig,ax = plt.subplots(3,3)

augmentation_applied_list=[]
idx_list=[]
for n in range(3):
    for m in range(3):
        augmentation_applied=random.sample(augmentations_list, 3)
        augmentation_applied_list.append(augmentation_applied)
        img_augm = apply_augment(img, augmentation_applied)
        ax[n,m].axis('off')
        ax[n,m].set_title("["+str(n)+","+str(m)+"]", fontweight='bold')
        ax[n,m].imshow(img_augm)
        idx_list.append([n,m])

for n in range(len(idx_list)):
    print("Image "+str(idx_list[n])+": "+str(augmentation_applied_list[n]))